# Classification
### Step 1: resize/crop images
Requires having run: `beescrape.py`
Depends on yolov5 and `shutil`

In [1]:
import os
from os import listdir
from shutil import copyfile
import shutil
from PIL import Image
import skimage
import random
import numpy as np

# Directories
DATA_DIR = '/home/wilber/Documents/RESEARCH/research/beespotter'
OUT_DIR = '/m2docs/res/data'
CROPPED_PATH = '/m2docs/res/cropped_imgs'
cropped_files = listdir(CROPPED_PATH)

%rm -R /m2docs/res/data/*

# Args:
VAL_SIZE = 128
TEST_SIZE = 128
preprocess = True
resize = True
length = 512
size = (length, length) # (512, 512)

# Species with >= 1000 images:
classes = ['Apis_mellifera','Bombus_impatiens','Bombus_auricomus','Bombus_bimaculatus','Bombus_griseocollis']

# from https://gist.github.com/Prasad9/28f6a2df8e8d463c6ddd040f4f6a028a
noise_modes = [None,'salt','pepper','s&p']
NOISE_AMOUNT = .01 # default amount
def add_noise(img, mode, noise_amount = .03):
    if mode is not None:
        gimg = skimage.util.random_noise(img, mode = mode, amount = random.uniform(0,noise_amount))
        return gimg
    else:
        print("oops, you shouldn't see this")

total_skipped = 0
for label in classes:  # for each type of bee
    skipnum = 0
    classID = classes.index(label)
    
    input_path = os.path.join(DATA_DIR, label)
    im_list = os.listdir(input_path)
    random.shuffle(im_list)
    
    # Attempt to create directories:
    if not os.path.exists(OUT_DIR + '/train/' + label):
        os.makedirs(OUT_DIR + '/train/' + label)
    if not os.path.exists(OUT_DIR + '/test/' + label):
        os.makedirs(OUT_DIR + '/test/' + label)
    if not os.path.exists(OUT_DIR + '/valid/' + label):
        os.makedirs(OUT_DIR + '/valid/' + label)
        
    train_path = os.path.join(OUT_DIR,'train/'+ label + '/')
    test_path  = os.path.join(OUT_DIR,'test/' + label + '/')
    valid_path = os.path.join(OUT_DIR,'valid/' + label + '/')
    
    index = 0
    for img in im_list:
        if img in cropped_files:
            pic = Image.open(os.path.join(CROPPED_PATH,img))
            
            if resize: #resize image
                out = pic.resize(size)
            else:
                out = pic
            
            if index < VAL_SIZE:
                out.save(os.path.join(valid_path,img))
            elif index < VAL_SIZE + TEST_SIZE:
                out.save(os.path.join(test_path,img))
            else:    #training set, rotate
                out.save(os.path.join(train_path,img.replace('.jpg','-0.jpg')))
                if preprocess:
                    for rot,mode,ext in zip([random.randint(0,359),random.randint(0,359),random.randint(0,359)],[random.choice(noise_modes),random.choice(noise_modes),random.choice(noise_modes)],[1,2,3]):
                        print(' ',img,rot,mode,ext,pic.mode)
                        if mode is not None:
                            Image.fromarray((add_noise(np.array(out.rotate(rot)),mode,NOISE_AMOUNT)*255).astype(np.uint8),pic.mode).save(os.path.join(train_path,img.replace('.jpg','-'+str(ext)+'.jpg')))
                        else:
                            out.rotate(rot).save(os.path.join(train_path,img.replace('.jpg','-'+str(ext)+'.jpg')))
                        
                
        else:
            skipnum += 1
            index -= 1
            pass
        index += 1
    print(label, "skipped", skipnum)
    print(label, "total", index)
    
    total_skipped += skipnum
    
print("Images moved and rotated. Skipped ", total_skipped)

  8269-1.jpg 310 None 1 RGB
  8269-1.jpg 115 pepper 2 RGB
  8269-1.jpg 120 s&p 3 RGB
  5775-5.jpg 249 pepper 1 RGB
  5775-5.jpg 133 salt 2 RGB
  5775-5.jpg 288 pepper 3 RGB
  5951-7.jpg 239 salt 1 RGB
  5951-7.jpg 9 None 2 RGB
  5951-7.jpg 80 None 3 RGB
  7092-1.jpg 143 None 1 RGB
  7092-1.jpg 21 s&p 2 RGB
  7092-1.jpg 165 None 3 RGB
  6606-1.jpg 150 salt 1 RGB
  6606-1.jpg 184 salt 2 RGB
  6606-1.jpg 173 s&p 3 RGB
  2919-4.jpg 93 salt 1 RGB
  2919-4.jpg 127 None 2 RGB
  2919-4.jpg 57 s&p 3 RGB
  6310-1.jpg 229 s&p 1 RGB
  6310-1.jpg 225 None 2 RGB
  6310-1.jpg 11 None 3 RGB
  2848-1.jpg 153 s&p 1 RGB
  2848-1.jpg 150 None 2 RGB
  2848-1.jpg 13 salt 3 RGB
  9272-6.jpg 159 None 1 RGB
  9272-6.jpg 341 None 2 RGB
  9272-6.jpg 239 salt 3 RGB
  977-1.jpg 277 None 1 RGB
  977-1.jpg 171 s&p 2 RGB
  977-1.jpg 101 None 3 RGB
  6581-1.jpg 199 None 1 RGB
  6581-1.jpg 354 s&p 2 RGB
  6581-1.jpg 226 pepper 3 RGB
  7885-4.jpg 32 None 1 RGB
  7885-4.jpg 167 pepper 2 RGB
  7885-4.jpg 71 salt 3 RGB
  1

  1820-3.jpg 278 None 1 RGB
  1820-3.jpg 190 pepper 2 RGB
  1820-3.jpg 110 s&p 3 RGB
  5031-2.jpg 73 s&p 1 RGB
  5031-2.jpg 61 None 2 RGB
  5031-2.jpg 120 None 3 RGB
  7415-1.jpg 226 None 1 RGB
  7415-1.jpg 159 None 2 RGB
  7415-1.jpg 279 pepper 3 RGB
  5809-1.jpg 324 s&p 1 RGB
  5809-1.jpg 242 None 2 RGB
  5809-1.jpg 44 s&p 3 RGB
  5135-4.jpg 348 None 1 RGB
  5135-4.jpg 346 None 2 RGB
  5135-4.jpg 87 pepper 3 RGB
  6521-12.jpg 312 pepper 1 RGB
  6521-12.jpg 15 s&p 2 RGB
  6521-12.jpg 244 s&p 3 RGB
  8927-1.jpg 231 s&p 1 RGB
  8927-1.jpg 17 None 2 RGB
  8927-1.jpg 7 s&p 3 RGB
  7195-1.jpg 198 None 1 RGB
  7195-1.jpg 195 pepper 2 RGB
  7195-1.jpg 97 salt 3 RGB
  6580-1.jpg 36 None 1 RGB
  6580-1.jpg 300 pepper 2 RGB
  6580-1.jpg 213 None 3 RGB
  1966-1.jpg 191 salt 1 RGB
  1966-1.jpg 12 pepper 2 RGB
  1966-1.jpg 273 None 3 RGB
  6816-1.jpg 355 pepper 1 RGB
  6816-1.jpg 262 None 2 RGB
  6816-1.jpg 304 s&p 3 RGB
  5156-1.jpg 198 s&p 1 RGB
  5156-1.jpg 199 salt 2 RGB
  5156-1.jpg 131 salt 

  9210-1.jpg 300 None 2 RGB
  9210-1.jpg 15 salt 3 RGB
  5704-9.jpg 34 s&p 1 RGB
  5704-9.jpg 190 None 2 RGB
  5704-9.jpg 193 None 3 RGB
  6522-2.jpg 306 pepper 1 RGB
  6522-2.jpg 212 salt 2 RGB
  6522-2.jpg 22 pepper 3 RGB
  3382-1.jpg 105 pepper 1 RGB
  3382-1.jpg 230 s&p 2 RGB
  3382-1.jpg 119 s&p 3 RGB
  5140-2.jpg 263 None 1 RGB
  5140-2.jpg 192 s&p 2 RGB
  5140-2.jpg 83 salt 3 RGB
  8795-1.jpg 274 s&p 1 RGB
  8795-1.jpg 265 s&p 2 RGB
  8795-1.jpg 229 None 3 RGB
  4397-1.jpg 287 s&p 1 RGB
  4397-1.jpg 7 s&p 2 RGB
  4397-1.jpg 26 s&p 3 RGB
  1634-1.jpg 85 None 1 RGB
  1634-1.jpg 122 s&p 2 RGB
  1634-1.jpg 243 s&p 3 RGB
  6331-1.jpg 29 s&p 1 RGB
  6331-1.jpg 282 s&p 2 RGB
  6331-1.jpg 324 None 3 RGB
  6523-12.jpg 105 s&p 1 RGB
  6523-12.jpg 244 pepper 2 RGB
  6523-12.jpg 143 pepper 3 RGB
  7965-1.jpg 219 salt 1 RGB
  7965-1.jpg 276 pepper 2 RGB
  7965-1.jpg 191 salt 3 RGB
  5486-1.jpg 63 pepper 1 RGB
  5486-1.jpg 286 None 2 RGB
  5486-1.jpg 358 salt 3 RGB
  6519-2.jpg 302 None 1 RGB

  7630-1.jpg 271 s&p 3 RGB
  285-1.jpg 350 None 1 RGB
  285-1.jpg 118 None 2 RGB
  285-1.jpg 126 salt 3 RGB
  1639-1.jpg 113 s&p 1 RGB
  1639-1.jpg 254 s&p 2 RGB
  1639-1.jpg 94 pepper 3 RGB
  9597-1.jpg 98 pepper 1 RGB
  9597-1.jpg 315 None 2 RGB
  9597-1.jpg 226 None 3 RGB
  4278-1.jpg 110 pepper 1 RGB
  4278-1.jpg 207 None 2 RGB
  4278-1.jpg 317 pepper 3 RGB
  4023-1.jpg 269 pepper 1 RGB
  4023-1.jpg 152 salt 2 RGB
  4023-1.jpg 214 None 3 RGB
  4160-2.jpg 8 None 1 RGB
  4160-2.jpg 44 salt 2 RGB
  4160-2.jpg 86 pepper 3 RGB
  8127-1.jpg 109 None 1 RGB
  8127-1.jpg 323 None 2 RGB
  8127-1.jpg 229 pepper 3 RGB
  6314-3.jpg 34 s&p 1 RGB
  6314-3.jpg 133 None 2 RGB
  6314-3.jpg 337 salt 3 RGB
  7773-1.jpg 218 pepper 1 RGB
  7773-1.jpg 204 salt 2 RGB
  7773-1.jpg 92 None 3 RGB
  5827-2.jpg 156 salt 1 RGB
  5827-2.jpg 38 pepper 2 RGB
  5827-2.jpg 221 None 3 RGB
  9492-1.jpg 251 pepper 1 RGB
  9492-1.jpg 298 None 2 RGB
  9492-1.jpg 13 salt 3 RGB
  7166-1.jpg 142 pepper 1 RGB
  7166-1.jpg 35

  871-1.jpg 337 s&p 2 RGB
  871-1.jpg 48 salt 3 RGB
  575-2.jpg 191 None 1 RGB
  575-2.jpg 44 None 2 RGB
  575-2.jpg 126 s&p 3 RGB
  4805-1.jpg 140 salt 1 RGB
  4805-1.jpg 323 s&p 2 RGB
  4805-1.jpg 194 salt 3 RGB
  4623-1.jpg 310 s&p 1 RGB
  4623-1.jpg 91 salt 2 RGB
  4623-1.jpg 149 salt 3 RGB
  5614-1.jpg 341 s&p 1 RGB
  5614-1.jpg 131 pepper 2 RGB
  5614-1.jpg 204 pepper 3 RGB
  7956-1.jpg 93 pepper 1 RGB
  7956-1.jpg 251 None 2 RGB
  7956-1.jpg 156 salt 3 RGB
  7866-2.jpg 20 None 1 RGB
  7866-2.jpg 92 salt 2 RGB
  7866-2.jpg 341 s&p 3 RGB
  3505-2.jpg 21 pepper 1 RGB
  3505-2.jpg 104 pepper 2 RGB
  3505-2.jpg 130 None 3 RGB
  1611-1.jpg 227 pepper 1 RGB
  1611-1.jpg 14 None 2 RGB
  1611-1.jpg 262 pepper 3 RGB
  5999-5.jpg 69 None 1 RGB
  5999-5.jpg 85 salt 2 RGB
  5999-5.jpg 54 s&p 3 RGB
  2013-1.jpg 15 None 1 RGB
  2013-1.jpg 22 pepper 2 RGB
  2013-1.jpg 32 salt 3 RGB
  8215-1.jpg 195 pepper 1 RGB
  8215-1.jpg 124 salt 2 RGB
  8215-1.jpg 165 s&p 3 RGB
  3545-3.jpg 217 pepper 1 RGB

...

  6377-1.jpg 276 None 1 RGB
  6377-1.jpg 134 s&p 2 RGB
  6377-1.jpg 129 s&p 3 RGB
  4263-3.jpg 355 None 1 RGB
  4263-3.jpg 2 pepper 2 RGB
  4263-3.jpg 112 pepper 3 RGB
  3740-1.jpg 294 s&p 1 RGB
  3740-1.jpg 25 pepper 2 RGB
  3740-1.jpg 6 s&p 3 RGB
  6240-6.jpg 193 None 1 RGB
  6240-6.jpg 178 None 2 RGB
  6240-6.jpg 104 None 3 RGB
  684-2.jpg 13 pepper 1 RGB
  684-2.jpg 306 s&p 2 RGB
  684-2.jpg 335 None 3 RGB
  4861-1.jpg 67 salt 1 RGB
  4861-1.jpg 235 salt 2 RGB
  4861-1.jpg 10 pepper 3 RGB
  6234-5.jpg 242 None 1 RGB
  6234-5.jpg 10 None 2 RGB
  6234-5.jpg 236 None 3 RGB
  5856-1.jpg 316 salt 1 RGB
  5856-1.jpg 242 salt 2 RGB
  5856-1.jpg 184 salt 3 RGB
  5924-4.jpg 130 s&p 1 RGB
  5924-4.jpg 359 s&p 2 RGB
  5924-4.jpg 34 None 3 RGB
  519-2.jpg 74 s&p 1 RGB
  519-2.jpg 180 salt 2 RGB
  519-2.jpg 118 pepper 3 RGB
  7711-1.jpg 61 salt 1 RGB
  7711-1.jpg 81 None 2 RGB
  7711-1.jpg 235 salt 3 RGB
  5871-1.jpg 42 pepper 1 RGB
  5871-1.jpg 302 s&p 2 RGB
  5871-1.jpg 357 salt 3 RGB
  8830-

  8899-2.jpg 236 None 1 RGB
  8899-2.jpg 25 s&p 2 RGB
  8899-2.jpg 186 s&p 3 RGB
  2997-2.jpg 262 None 1 RGB
  2997-2.jpg 189 s&p 2 RGB
  2997-2.jpg 198 None 3 RGB
  6838-5.jpg 343 pepper 1 RGB
  6838-5.jpg 303 salt 2 RGB
  6838-5.jpg 46 s&p 3 RGB
  4211-2.jpg 351 s&p 1 RGB
  4211-2.jpg 84 s&p 2 RGB
  4211-2.jpg 58 None 3 RGB
  8814-2.jpg 139 s&p 1 RGB
  8814-2.jpg 314 s&p 2 RGB
  8814-2.jpg 212 None 3 RGB
  8930-2.jpg 203 s&p 1 RGB
  8930-2.jpg 314 pepper 2 RGB
  8930-2.jpg 60 s&p 3 RGB
  6780-6.jpg 54 s&p 1 RGB
  6780-6.jpg 9 None 2 RGB
  6780-6.jpg 155 salt 3 RGB
  4263-2.jpg 229 None 1 RGB
  4263-2.jpg 158 s&p 2 RGB
  4263-2.jpg 196 s&p 3 RGB
  7371-3.jpg 193 s&p 1 RGB
  7371-3.jpg 354 None 2 RGB
  7371-3.jpg 77 salt 3 RGB
  5469-1.jpg 206 s&p 1 RGB
  5469-1.jpg 248 None 2 RGB
  5469-1.jpg 86 None 3 RGB
  8583-11.jpg 240 pepper 1 RGB
  8583-11.jpg 314 s&p 2 RGB
  8583-11.jpg 182 None 3 RGB
  6837-3.jpg 271 s&p 1 RGB
  6837-3.jpg 274 s&p 2 RGB
  6837-3.jpg 111 salt 3 RGB
  3613-3.jp

  8391-1.jpg 114 None 3 RGB
  393-1.jpg 181 None 1 RGB
  393-1.jpg 220 salt 2 RGB
  393-1.jpg 176 salt 3 RGB
  7751-8.jpg 257 s&p 1 RGB
  7751-8.jpg 355 pepper 2 RGB
  7751-8.jpg 164 salt 3 RGB
  5692-1.jpg 200 s&p 1 RGB
  5692-1.jpg 343 s&p 2 RGB
  5692-1.jpg 19 s&p 3 RGB
  7078-4.jpg 328 None 1 RGB
  7078-4.jpg 150 salt 2 RGB
  7078-4.jpg 51 pepper 3 RGB
  7236-2.jpg 231 pepper 1 RGB
  7236-2.jpg 192 None 2 RGB
  7236-2.jpg 194 s&p 3 RGB
  7590-1.jpg 297 s&p 1 RGB
  7590-1.jpg 197 s&p 2 RGB
  7590-1.jpg 62 salt 3 RGB
  3119-1.jpg 136 s&p 1 RGB
  3119-1.jpg 302 pepper 2 RGB
  3119-1.jpg 178 pepper 3 RGB
  8639-3.jpg 4 pepper 1 RGB
  8639-3.jpg 224 None 2 RGB
  8639-3.jpg 294 s&p 3 RGB
  5126-1.jpg 233 s&p 1 RGB
  5126-1.jpg 326 pepper 2 RGB
  5126-1.jpg 281 None 3 RGB
  8501-2.jpg 104 s&p 1 RGB
  8501-2.jpg 157 s&p 2 RGB
  8501-2.jpg 344 salt 3 RGB
  2538-1.jpg 95 None 1 RGB
  2538-1.jpg 312 s&p 2 RGB
  2538-1.jpg 127 None 3 RGB
  6743-1.jpg 3 pepper 1 RGB
  6743-1.jpg 19 pepper 2 RGB

  9250-1.jpg 39 salt 3 RGB
  6289-1.jpg 78 None 1 RGB
  6289-1.jpg 87 None 2 RGB
  6289-1.jpg 204 s&p 3 RGB
  4379-1.jpg 337 None 1 RGB
  4379-1.jpg 31 s&p 2 RGB
  4379-1.jpg 37 pepper 3 RGB
  5879-4.jpg 194 None 1 RGB
  5879-4.jpg 20 salt 2 RGB
  5879-4.jpg 105 None 3 RGB
  9415-1.jpg 311 salt 1 RGB
  9415-1.jpg 109 pepper 2 RGB
  9415-1.jpg 332 s&p 3 RGB
  2396-1.jpg 43 salt 1 RGB
  2396-1.jpg 147 None 2 RGB
  2396-1.jpg 149 None 3 RGB
  5566-2.jpg 185 pepper 1 RGB
  5566-2.jpg 84 salt 2 RGB
  5566-2.jpg 180 s&p 3 RGB
  5922-2.jpg 303 pepper 1 RGB
  5922-2.jpg 159 salt 2 RGB
  5922-2.jpg 174 s&p 3 RGB
  6885-1.jpg 267 s&p 1 RGB
  6885-1.jpg 85 None 2 RGB
  6885-1.jpg 146 pepper 3 RGB
  8838-2.jpg 238 salt 1 RGB
  8838-2.jpg 219 salt 2 RGB
  8838-2.jpg 0 pepper 3 RGB
  1740-2.jpg 146 salt 1 RGB
  1740-2.jpg 85 salt 2 RGB
  1740-2.jpg 252 salt 3 RGB
  5858-3.jpg 137 s&p 1 RGB
  5858-3.jpg 216 s&p 2 RGB
  5858-3.jpg 353 s&p 3 RGB
  3105-1.jpg 283 None 1 RGB
  3105-1.jpg 291 pepper 2 RGB

  6871-3.jpg 246 None 1 RGB
  6871-3.jpg 294 None 2 RGB
  6871-3.jpg 145 None 3 RGB
  3707-1.jpg 321 pepper 1 RGB
  3707-1.jpg 9 s&p 2 RGB
  3707-1.jpg 152 salt 3 RGB
  6161-2.jpg 41 s&p 1 RGB
  6161-2.jpg 333 s&p 2 RGB
  6161-2.jpg 274 s&p 3 RGB
  8614-1.jpg 1 salt 1 RGB
  8614-1.jpg 118 None 2 RGB
  8614-1.jpg 117 salt 3 RGB
  5137-5.jpg 138 s&p 1 RGB
  5137-5.jpg 24 salt 2 RGB
  5137-5.jpg 139 s&p 3 RGB
  8943-1.jpg 80 None 1 RGB
  8943-1.jpg 9 None 2 RGB
  8943-1.jpg 303 pepper 3 RGB
  7066-1.jpg 168 s&p 1 RGB
  7066-1.jpg 31 salt 2 RGB
  7066-1.jpg 34 pepper 3 RGB
  4978-1.jpg 184 pepper 1 RGB
  4978-1.jpg 203 pepper 2 RGB
  4978-1.jpg 45 s&p 3 RGB
  4388-1.jpg 144 None 1 RGB
  4388-1.jpg 305 salt 2 RGB
  4388-1.jpg 23 salt 3 RGB
  8704-1.jpg 272 pepper 1 RGB
  8704-1.jpg 145 s&p 2 RGB
  8704-1.jpg 225 None 3 RGB
  7306-3.jpg 199 s&p 1 RGB
  7306-3.jpg 138 salt 2 RGB
  7306-3.jpg 180 salt 3 RGB
  6105-1.jpg 157 salt 1 RGB
  6105-1.jpg 219 pepper 2 RGB
  6105-1.jpg 53 salt 3 RGB
  

In [2]:
# Flip images over axes to even out training sets:
import fnmatch
import os
from PIL import Image
import random
import skimage

classes = ['Apis_mellifera','Bombus_impatiens','Bombus_auricomus','Bombus_bimaculatus','Bombus_griseocollis']
ABS_PATH_TRAIN = '/m2docs/res/data/train'
MULTIPLIER = 1.2 # how many times more images need to be created: mult * len(largest class)

c_count = [len(fnmatch.filter(os.listdir(os.path.join(ABS_PATH_TRAIN,c_name)), '*')) for c_name in classes]
print(classes)
print(c_count)

# add 25% more images to largest class,
# and bring the others to the same count
target = max(c_count) * 1.2
print("Target count: n =",target)

noise_modes = [None,'salt','pepper','s&p']
NOISE_AMOUNT = .01 # default amount
def add_noise(img, mode, noise_amount = .03):
    if mode is not None:
        gimg = skimage.util.random_noise(img, mode = mode, amount = random.uniform(0, noise_amount))
        return gimg
    else:
        print("oops, you shouldn't see this")

for c_name in classes:
    current = c_count[classes.index(c_name)]
    print(c_name, current, "->", target)
    im_list = os.listdir(os.path.join(ABS_PATH_TRAIN, c_name))
    while current < target:
        filename = random.choice(im_list)
        print(filename)
        out = Image.open(os.path.join(ABS_PATH_TRAIN, c_name, filename))
        
        rot = random.randint(0,359)
        mode = random.choice(noise_modes)
        print(filename,rot,mode,out.mode)
        print(filename.replace('.jpg','-x'+str(current)+'.jpg'))
        if mode is not None:
            Image.fromarray((add_noise(np.array(out.rotate(rot)),mode,NOISE_AMOUNT)*255).astype(np.uint8),pic.mode).save(os.path.join(ABS_PATH_TRAIN, c_name, filename.replace('.jpg','-x'+str(current)+'.jpg')))
        else:
            out.rotate(rot).save(os.path.join(ABS_PATH_TRAIN, c_name, filename.replace('.jpg','-x'+str(current)+'.jpg')))
        current += 1

c_count = [len(fnmatch.filter(os.listdir(os.path.join(ABS_PATH_TRAIN,c_name)), '*')) for c_name in classes]
print(c_count)


['Apis_mellifera', 'Bombus_impatiens', 'Bombus_auricomus', 'Bombus_bimaculatus', 'Bombus_griseocollis']
[3844, 7044, 1872, 3420, 6300]
Target count: n = 8452.8
Apis_mellifera 3844 -> 8452.8
3316-1-2.jpg
3316-1-2.jpg 342 None RGB
3316-1-2-x3844.jpg
230-1-0.jpg
230-1-0.jpg 278 pepper RGB
230-1-0-x3845.jpg
6522-2-2.jpg
6522-2-2.jpg 267 pepper RGB
6522-2-2-x3846.jpg
8127-1-0.jpg
8127-1-0.jpg 245 salt RGB
8127-1-0-x3847.jpg
8245-2-3.jpg
8245-2-3.jpg 43 s&p RGB
8245-2-3-x3848.jpg
5183-5-3.jpg
5183-5-3.jpg 63 salt RGB
5183-5-3-x3849.jpg
4226-2-0.jpg
4226-2-0.jpg 48 salt RGB
4226-2-0-x3850.jpg
8099-3-0.jpg
8099-3-0.jpg 239 pepper RGB
8099-3-0-x3851.jpg
4804-1-3.jpg
4804-1-3.jpg 356 salt RGB
4804-1-3-x3852.jpg
6517-4-1.jpg
6517-4-1.jpg 139 salt RGB
6517-4-1-x3853.jpg
7860-2-3.jpg
7860-2-3.jpg 291 pepper RGB
7860-2-3-x3854.jpg
2480-2-2.jpg
2480-2-2.jpg 128 s&p RGB
2480-2-2-x3855.jpg
6280-4-3.jpg
6280-4-3.jpg 83 pepper RGB
6280-4-3-x3856.jpg
67-2-3.jpg
67-2-3.jpg 152 pepper RGB
67-2-3-x3857.jpg
1

7456-2-0.jpg
7456-2-0.jpg 143 pepper RGB
7456-2-0-x3990.jpg
8079-2-3.jpg
8079-2-3.jpg 330 s&p RGB
8079-2-3-x3991.jpg
5866-4-3.jpg
5866-4-3.jpg 263 salt RGB
5866-4-3-x3992.jpg
5135-2-1.jpg
5135-2-1.jpg 222 salt RGB
5135-2-1-x3993.jpg
5775-5-1.jpg
5775-5-1.jpg 358 s&p RGB
5775-5-1-x3994.jpg
4023-1-0.jpg
4023-1-0.jpg 235 salt RGB
4023-1-0-x3995.jpg
4297-2-1.jpg
4297-2-1.jpg 16 pepper RGB
4297-2-1-x3996.jpg
7287-5-3.jpg
7287-5-3.jpg 142 s&p RGB
7287-5-3-x3997.jpg
6312-4-0.jpg
6312-4-0.jpg 232 pepper RGB
6312-4-0-x3998.jpg
7630-1-0.jpg
7630-1-0.jpg 194 s&p RGB
7630-1-0-x3999.jpg
8194-2-0.jpg
8194-2-0.jpg 7 None RGB
8194-2-0-x4000.jpg
4384-1-2.jpg
4384-1-2.jpg 325 pepper RGB
4384-1-2-x4001.jpg
2729-1-1.jpg
2729-1-1.jpg 300 salt RGB
2729-1-1-x4002.jpg
6351-3-0.jpg
6351-3-0.jpg 196 pepper RGB
6351-3-0-x4003.jpg
7894-1-2.jpg
7894-1-2.jpg 209 pepper RGB
7894-1-2-x4004.jpg
2528-2-2.jpg
2528-2-2.jpg 47 salt RGB
2528-2-2-x4005.jpg
8147-1-3.jpg
8147-1-3.jpg 126 salt RGB
8147-1-3-x4006.jpg
8105-1-0.j

1419-1-2.jpg
1419-1-2.jpg 311 pepper RGB
1419-1-2-x4134.jpg
4278-1-1.jpg
4278-1-1.jpg 186 salt RGB
4278-1-1-x4135.jpg
5967-4-2.jpg
5967-4-2.jpg 148 pepper RGB
5967-4-2-x4136.jpg
1987-1-0.jpg
1987-1-0.jpg 91 None RGB
1987-1-0-x4137.jpg
7087-1-2.jpg
7087-1-2.jpg 85 pepper RGB
7087-1-2-x4138.jpg
6327-1-2.jpg
6327-1-2.jpg 220 None RGB
6327-1-2-x4139.jpg
4027-2-3.jpg
4027-2-3.jpg 320 pepper RGB
4027-2-3-x4140.jpg
6310-6-0.jpg
6310-6-0.jpg 285 s&p RGB
6310-6-0-x4141.jpg
5827-3-0.jpg
5827-3-0.jpg 257 pepper RGB
5827-3-0-x4142.jpg
6404-2-3.jpg
6404-2-3.jpg 32 s&p RGB
6404-2-3-x4143.jpg
4277-2-2.jpg
4277-2-2.jpg 244 pepper RGB
4277-2-2-x4144.jpg
6280-3-3.jpg
6280-3-3.jpg 358 salt RGB
6280-3-3-x4145.jpg
4019-2-0.jpg
4019-2-0.jpg 12 salt RGB
4019-2-0-x4146.jpg
1815-1-0.jpg
1815-1-0.jpg 133 pepper RGB
1815-1-0-x4147.jpg
6521-9-2.jpg
6521-9-2.jpg 32 None RGB
6521-9-2-x4148.jpg
6816-4-2.jpg
6816-4-2.jpg 150 salt RGB
6816-4-2-x4149.jpg
9189-1-2.jpg
9189-1-2.jpg 53 salt RGB
9189-1-2-x4150.jpg
5171-1-2

6280-5-2.jpg
6280-5-2.jpg 298 None RGB
6280-5-2-x4282.jpg
2101-1-3.jpg
2101-1-3.jpg 48 pepper RGB
2101-1-3-x4283.jpg
4277-2-0.jpg
4277-2-0.jpg 179 None RGB
4277-2-0-x4284.jpg
7870-2-3.jpg
7870-2-3.jpg 258 None RGB
7870-2-3-x4285.jpg
5804-2-1.jpg
5804-2-1.jpg 120 s&p RGB
5804-2-1-x4286.jpg
8252-1-2.jpg
8252-1-2.jpg 285 None RGB
8252-1-2-x4287.jpg
6523-6-3.jpg
6523-6-3.jpg 103 pepper RGB
6523-6-3-x4288.jpg
8063-1-0.jpg
8063-1-0.jpg 39 None RGB
8063-1-0-x4289.jpg
5720-1-0.jpg
5720-1-0.jpg 113 s&p RGB
5720-1-0-x4290.jpg
6577-1-3.jpg
6577-1-3.jpg 143 salt RGB
6577-1-3-x4291.jpg
6517-1-0.jpg
6517-1-0.jpg 48 salt RGB
6517-1-0-x4292.jpg
952-1-0.jpg
952-1-0.jpg 80 s&p RGB
952-1-0-x4293.jpg
8127-1-0.jpg
8127-1-0.jpg 141 pepper RGB
8127-1-0-x4294.jpg
9570-2-2.jpg
9570-2-2.jpg 124 pepper RGB
9570-2-2-x4295.jpg
6499-2-0.jpg
6499-2-0.jpg 104 None RGB
6499-2-0-x4296.jpg
2001-1-0.jpg
2001-1-0.jpg 32 pepper RGB
2001-1-0-x4297.jpg
7877-4-2.jpg
7877-4-2.jpg 174 salt RGB
7877-4-2-x4298.jpg
5701-1-0.jpg
57

7863-3-0.jpg
7863-3-0.jpg 176 s&p RGB
7863-3-0-x4427.jpg
7856-1-1.jpg
7856-1-1.jpg 306 pepper RGB
7856-1-1-x4428.jpg
8269-2-1.jpg
8269-2-1.jpg 31 s&p RGB
8269-2-1-x4429.jpg
8119-1-0.jpg
8119-1-0.jpg 101 None RGB
8119-1-0-x4430.jpg
5968-2-0.jpg
5968-2-0.jpg 357 s&p RGB
5968-2-0-x4431.jpg
8499-2-2.jpg
8499-2-2.jpg 189 s&p RGB
8499-2-2-x4432.jpg
6330-1-3.jpg
6330-1-3.jpg 333 s&p RGB
6330-1-3-x4433.jpg
7101-3-3.jpg
7101-3-3.jpg 309 pepper RGB
7101-3-3-x4434.jpg
9210-1-0.jpg
9210-1-0.jpg 48 pepper RGB
9210-1-0-x4435.jpg
2607-1-3.jpg
2607-1-3.jpg 275 None RGB
2607-1-3-x4436.jpg
9030-3-3.jpg
9030-3-3.jpg 284 s&p RGB
9030-3-3-x4437.jpg
934-1-2.jpg
934-1-2.jpg 358 s&p RGB
934-1-2-x4438.jpg
5967-4-3.jpg
5967-4-3.jpg 1 s&p RGB
5967-4-3-x4439.jpg
7868-1-3.jpg
7868-1-3.jpg 124 None RGB
7868-1-3-x4440.jpg
1141-1-3.jpg
1141-1-3.jpg 310 s&p RGB
1141-1-3-x4441.jpg
5839-1-0.jpg
5839-1-0.jpg 19 s&p RGB
5839-1-0-x4442.jpg
5171-2-0.jpg
5171-2-0.jpg 247 s&p RGB
5171-2-0-x4443.jpg
2492-1-2.jpg
2492-1-2.jpg 1

...

4205-2-0.jpg
4205-2-0.jpg 145 None RGB
4205-2-0-x7836.jpg
4172-2-3.jpg
4172-2-3.jpg 102 None RGB
4172-2-3-x7837.jpg
5469-1-0.jpg
5469-1-0.jpg 115 s&p RGB
5469-1-0-x7838.jpg
3251-1-2.jpg
3251-1-2.jpg 119 None RGB
3251-1-2-x7839.jpg
8296-3-0.jpg
8296-3-0.jpg 336 None RGB
8296-3-0-x7840.jpg
8583-7-2.jpg
8583-7-2.jpg 26 salt RGB
8583-7-2-x7841.jpg
1177-2-0.jpg
1177-2-0.jpg 222 pepper RGB
1177-2-0-x7842.jpg
5807-2-1.jpg
5807-2-1.jpg 90 pepper RGB
5807-2-1-x7843.jpg
5124-2-1.jpg
5124-2-1.jpg 13 pepper RGB
5124-2-1-x7844.jpg
5886-1-2.jpg
5886-1-2.jpg 346 pepper RGB
5886-1-2-x7845.jpg
7320-1-0.jpg
7320-1-0.jpg 259 None RGB
7320-1-0-x7846.jpg
7487-3-3.jpg
7487-3-3.jpg 245 None RGB
7487-3-3-x7847.jpg
7376-3-2.jpg
7376-3-2.jpg 251 pepper RGB
7376-3-2-x7848.jpg
7451-2-1.jpg
7451-2-1.jpg 247 None RGB
7451-2-1-x7849.jpg
6922-3-2.jpg
6922-3-2.jpg 29 s&p RGB
6922-3-2-x7850.jpg
552-1-0.jpg
552-1-0.jpg 39 pepper RGB
552-1-0-x7851.jpg
6889-4-3.jpg
6889-4-3.jpg 189 s&p RGB
6889-4-3-x7852.jpg
5148-1-3.jpg


7236-2-1.jpg
7236-2-1.jpg 107 None RGB
7236-2-1-x7980.jpg
5391-1-2.jpg
5391-1-2.jpg 76 s&p RGB
5391-1-2-x7981.jpg
8546-2-3.jpg
8546-2-3.jpg 70 s&p RGB
8546-2-3-x7982.jpg
5566-1-3.jpg
5566-1-3.jpg 260 pepper RGB
5566-1-3-x7983.jpg
4198-5-3.jpg
4198-5-3.jpg 9 salt RGB
4198-5-3-x7984.jpg
4379-1-2.jpg
4379-1-2.jpg 297 salt RGB
4379-1-2-x7985.jpg
6290-1-1.jpg
6290-1-1.jpg 178 salt RGB
6290-1-1-x7986.jpg
5807-6-2.jpg
5807-6-2.jpg 53 pepper RGB
5807-6-2-x7987.jpg
6843-5-1.jpg
6843-5-1.jpg 245 salt RGB
6843-5-1-x7988.jpg
6016-1-1.jpg
6016-1-1.jpg 357 salt RGB
6016-1-1-x7989.jpg
5269-3-0.jpg
5269-3-0.jpg 317 s&p RGB
5269-3-0-x7990.jpg
7249-1-2.jpg
7249-1-2.jpg 153 pepper RGB
7249-1-2-x7991.jpg
8267-1-3.jpg
8267-1-3.jpg 197 None RGB
8267-1-3-x7992.jpg
7454-2-0.jpg
7454-2-0.jpg 317 salt RGB
7454-2-0-x7993.jpg
5913-1-2.jpg
5913-1-2.jpg 220 None RGB
5913-1-2-x7994.jpg
6169-1-0.jpg
6169-1-0.jpg 45 None RGB
6169-1-0-x7995.jpg
235-2-2.jpg
235-2-2.jpg 210 None RGB
235-2-2-x7996.jpg
8918-2-3.jpg
8918-2-

3925-1-2.jpg
3925-1-2.jpg 163 None RGB
3925-1-2-x8122.jpg
8899-7-0.jpg
8899-7-0.jpg 89 pepper RGB
8899-7-0-x8123.jpg
9378-1-0.jpg
9378-1-0.jpg 209 pepper RGB
9378-1-0-x8124.jpg
1277-1-1.jpg
1277-1-1.jpg 142 s&p RGB
1277-1-1-x8125.jpg
805-1-1.jpg
805-1-1.jpg 147 s&p RGB
805-1-1-x8126.jpg
6841-2-0.jpg
6841-2-0.jpg 139 salt RGB
6841-2-0-x8127.jpg
4399-2-3.jpg
4399-2-3.jpg 278 pepper RGB
4399-2-3-x8128.jpg
6338-2-1.jpg
6338-2-1.jpg 33 pepper RGB
6338-2-1-x8129.jpg
7783-1-2.jpg
7783-1-2.jpg 11 salt RGB
7783-1-2-x8130.jpg
6780-6-1.jpg
6780-6-1.jpg 216 s&p RGB
6780-6-1-x8131.jpg
5722-2-1.jpg
5722-2-1.jpg 102 None RGB
5722-2-1-x8132.jpg
7066-2-2.jpg
7066-2-2.jpg 144 pepper RGB
7066-2-2-x8133.jpg
4988-2-2.jpg
4988-2-2.jpg 333 pepper RGB
4988-2-2-x8134.jpg
5155-1-2.jpg
5155-1-2.jpg 278 None RGB
5155-1-2-x8135.jpg
7076-2-0.jpg
7076-2-0.jpg 80 pepper RGB
7076-2-0-x8136.jpg
6955-2-3.jpg
6955-2-3.jpg 185 None RGB
6955-2-3-x8137.jpg
5269-3-0.jpg
5269-3-0.jpg 235 salt RGB
5269-3-0-x8138.jpg
5261-4-3.j

3005-4-3.jpg
3005-4-3.jpg 309 pepper RGB
3005-4-3-x8268.jpg
7086-1-1.jpg
7086-1-1.jpg 285 salt RGB
7086-1-1-x8269.jpg
4980-4-3.jpg
4980-4-3.jpg 62 salt RGB
4980-4-3-x8270.jpg
4797-1-0.jpg
4797-1-0.jpg 312 pepper RGB
4797-1-0-x8271.jpg
7430-5-2.jpg
7430-5-2.jpg 134 None RGB
7430-5-2-x8272.jpg
4926-3-2.jpg
4926-3-2.jpg 14 None RGB
4926-3-2-x8273.jpg
9116-5-3.jpg
9116-5-3.jpg 137 pepper RGB
9116-5-3-x8274.jpg
6144-3-1.jpg
6144-3-1.jpg 55 None RGB
6144-3-1-x8275.jpg
8502-5-0.jpg
8502-5-0.jpg 24 s&p RGB
8502-5-0-x8276.jpg
6338-1-3.jpg
6338-1-3.jpg 279 pepper RGB
6338-1-3-x8277.jpg
8293-4-0.jpg
8293-4-0.jpg 319 salt RGB
8293-4-0-x8278.jpg
1748-1-0.jpg
1748-1-0.jpg 238 pepper RGB
1748-1-0-x8279.jpg
9362-1-2.jpg
9362-1-2.jpg 145 None RGB
9362-1-2-x8280.jpg
6821-4-2.jpg
6821-4-2.jpg 10 s&p RGB
6821-4-2-x8281.jpg
6203-5-3.jpg
6203-5-3.jpg 220 salt RGB
6203-5-3-x8282.jpg
7783-4-2.jpg
7783-4-2.jpg 208 salt RGB
7783-4-2-x8283.jpg
5871-1-0.jpg
5871-1-0.jpg 341 None RGB
5871-1-0-x8284.jpg
6386-4-3.jp

8161-1-3.jpg
8161-1-3.jpg 237 None RGB
8161-1-3-x8412.jpg
1729-4-0.jpg
1729-4-0.jpg 178 salt RGB
1729-4-0-x8413.jpg
4267-3-3.jpg
4267-3-3.jpg 30 pepper RGB
4267-3-3-x8414.jpg
5192-3-1.jpg
5192-3-1.jpg 57 None RGB
5192-3-1-x8415.jpg
7535-1-3.jpg
7535-1-3.jpg 233 None RGB
7535-1-3-x8416.jpg
7124-4-2.jpg
7124-4-2.jpg 239 None RGB
7124-4-2-x8417.jpg
6209-3-2.jpg
6209-3-2.jpg 220 None RGB
6209-3-2-x8418.jpg
7323-1-3.jpg
7323-1-3.jpg 184 pepper RGB
7323-1-3-x8419.jpg
5261-4-3.jpg
5261-4-3.jpg 31 None RGB
5261-4-3-x8420.jpg
6155-1-0.jpg
6155-1-0.jpg 27 None RGB
6155-1-0-x8421.jpg
9251-1-2.jpg
9251-1-2.jpg 169 s&p RGB
9251-1-2-x8422.jpg
6731-4-1.jpg
6731-4-1.jpg 92 None RGB
6731-4-1-x8423.jpg
3249-2-1.jpg
3249-2-1.jpg 244 None RGB
3249-2-1-x8424.jpg
9055-2-3.jpg
9055-2-3.jpg 180 pepper RGB
9055-2-3-x8425.jpg
8710-3-0.jpg
8710-3-0.jpg 199 pepper RGB
8710-3-0-x8426.jpg
7944-2-3.jpg
7944-2-3.jpg 338 pepper RGB
7944-2-3-x8427.jpg
373-1-1.jpg
373-1-1.jpg 97 pepper RGB
373-1-1-x8428.jpg
6739-2-2.jpg

## Train classifier
Based on [this](https://nextjournal.com/gkoehler/pytorch-mnist) pytorch tutorial

Load images and train classifier.

In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import os, datetime, math

from torch.utils.tensorboard import SummaryWriter 
length = 512
tag = ':' + str(length) + 'x4_16-128'
print("running"+tag)
writer = SummaryWriter(log_dir = os.path.join('runs/',(datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + tag)))

classes = ['Apis_mellifera','Bombus_impatiens','Bombus_auricomus','Bombus_bimaculatus','Bombus_griseocollis']


ABS_PATH_TRAIN = '/m2docs/res/data/train'
ABS_PATH_VALID = '/m2docs/res/data/valid'
ABS_PATH_TEST = '/m2docs/res/data/test'

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def load(dir_name, batch_size, shuffle = False):
    return(
        #create a data loader
        torch.utils.data.DataLoader(
            datasets.ImageFolder(root = dir_name, transform = transforms.ToTensor()),
            batch_size = batch_size,
            num_workers = 2,
            shuffle = shuffle
        )
    )

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout2d()
        
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.conv3 = nn.Conv2d(32, 64, 5)
        self.conv4 = nn.Conv2d(64, 128, 5)
        
        self.fc1 = nn.Linear(128*28*28, length)
        self.fc2 = nn.Linear(length, 64)
        self.fc3 = nn.Linear(64, 5)

    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = F.relu(self.pool(self.conv3(x)))
        x = F.relu(self.pool(self.conv4(x)))
        #print(x.size())
        x = x.view(x.size(0),-1)
        #print(x.size())
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(self.fc3(x))
        return x

def validate(network,device,load_valid,criterion = nn.CrossEntropyLoss()):
    network.eval().to(device)
    correct = 0
    valid_loss = 0
    with torch.no_grad():
        for index, data in enumerate(load_valid, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            # forward + backward + optimize
            outputs = network(inputs)
            valid_loss += criterion(outputs, labels)

            # gather accuracy stats:
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum().item()
    accuracy = 100 * correct / len(load_valid.dataset)
    valid_loss = valid_loss / len(load_valid.dataset)
    return accuracy , valid_loss

def train(max_epochs = 50, min_epochs = 5, epoch_stretch = 5, train_path = ABS_PATH_TRAIN, valid_path = ABS_PATH_VALID, test_path = ABS_PATH_TEST, labels = classes, batch_size = 24):
    epochs = max_epochs
    class_names = labels
    num_classes = len(class_names)
    train_batch = batch_size
    test_batch = 128
    SAVE_PATH = '/m2docs/res/models'

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Device: {}".format(device))
    net = Net().to(device)
    print(net)

    criterion = nn.CrossEntropyLoss()
    #optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    optimizer = optim.Adam(net.parameters(), lr = 0.001)

    torch.manual_seed(417)
    
    load_train = load(train_path, batch_size, shuffle=True)
    load_valid = load(valid_path, batch_size, shuffle=True)
    load_test  = load(test_path, batch_size, shuffle=True)

    validation_accuracies = []
    best_epoch = 0
    epoch = 0
    while (epoch <= best_epoch + epoch_stretch or epoch < min_epochs) and epoch < max_epochs:
        net.train()
        run_loss = 0.0
        sum_loss = 0.0
        count = 0
        correct = 0.0
        categorical_correct = [0.0 for i in range(num_classes)]
        for index, data in enumerate(load_train, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # gather accuracy stats:
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum().item()
            for i in range(num_classes):
                categorical_correct[i] += ((predicted==i) == (labels==i)).float().sum().item()
                #print(classes[i],categorical_correct[i]/((index+1)*batch_size))
            
            # print statistics
            run_loss += loss.item()
            sum_loss += loss.item()
            count += 1 
            if index % 200 == 0:    # print every 200 mini-batches
                print('  Epoch: {} [{}/{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAcc: {:.6f}%'.format(
                    epoch, int(correct), (index + 1) * batch_size, len(load_train.dataset),
                    100. * index / len(load_train), loss.item(), 100. * correct / ((index+1) * batch_size)))
                run_loss = 0.0
        accuracy = 100. * correct / len(load_train.dataset)
        print('Epoch: {}\tLoss: {:.6f}\tAcc: {:.6f}'.format(
                epoch, sum_loss/count, accuracy))
        writer.add_scalar("Loss/train", sum_loss/count, epoch)
        writer.add_scalar("Acc/train", accuracy, epoch)
        
        modules_list = iter(net.named_modules())
        next(modules_list)
        for module in modules_list:
            try:
                writer.add_histogram("Model/"+module[0]+".weights", module[1].weight, epoch)
                writer.add_histogram("Model/"+module[0]+".bias", module[1].bias, epoch)
            except:
                pass
        # Categorical accuracy:
        for i in range(num_classes):
            writer.add_scalar("Acc/" + classes[i],categorical_correct[i]/len(load_train.dataset), epoch)
        torch.save(net.state_dict(), SAVE_PATH+"_progress")
        
        # get validation accuracy: 
        valid_acc, valid_loss = validate(net, device, load_valid, criterion)
        writer.add_scalar("Loss/valid", valid_loss, epoch)
        writer.add_scalar("Acc/valid", valid_acc, epoch)
        print('Validation: acc: {:.6f}%\tloss: {:.6f}'.format(
                valid_acc, valid_loss))
        validation_accuracies.append(valid_acc)
        best_epoch = validation_accuracies.index(max(validation_accuracies))
        print('Best: {} @ {:.6f}% -> epoch target {}'.format(best_epoch,validation_accuracies[best_epoch],max([best_epoch+epoch_stretch,min_epochs])))
        epoch += 1
        
    print('Done training.')
    torch.save(net.state_dict(), SAVE_PATH)

    prediction_list = torch.zeros(0,dtype=torch.long).to(device)
    label_list = torch.zeros(0,dtype=torch.long).to(device)
              
    with torch.no_grad():
        for data in load_test:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
              
            prediction_list = torch.cat([prediction_list, predicted.view(-1)])  
            label_list = torch.cat([label_list, labels.view(-1)])
    
    matrix = confusion_matrix(label_list.cpu().numpy(), prediction_list.cpu().numpy())
    print(matrix)
    fig, ax = plt.subplots()
    im = ax.imshow(matrix)
    # We want to show all ticks...
    ax.set_xticks(np.arange(len(classes)))
    ax.set_yticks(np.arange(len(classes)))
    # ... and label them with the respective list entries
    ax.set_xticklabels(classes)
    ax.set_yticklabels(classes)
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    for i in range(len(classes)):
        for j in range(len(classes)):
            text = ax.text(j, i, matrix[i, j],
                           ha="center", va="center", color="w")

    ax.set_title("Testing confusion matrix")

    plt.show()
    writer.add_figure('Testing/conf',fig)
    
    class_accuracy=100*matrix.diagonal() / matrix.sum(1)
    print(classes)
    print(class_accuracy)

train(max_epochs = 100, min_epochs = 70, epoch_stretch = 10, train_path = ABS_PATH_TRAIN, valid_path = ABS_PATH_VALID, test_path = ABS_PATH_TEST, labels = classes, batch_size = 32)

running:512x4_16-128
Device: cuda:0
Net(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout2d(p=0.5, inplace=False)
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=100352, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=5, bias=True)
)
  Epoch: 0 [8/32/42265 (0%)]	Loss: 1.594683	Acc: 25.000000%
  Epoch: 0 [1284/6432/42265 (15%)]	Loss: 1.610489	Acc: 19.962687%
  Epoch: 0 [2940/12832/42265 (30%)]	Loss: 1.344890	Acc: 22.911471%
  Epoch: 0 [4865/19232/42265 (45%)]	Loss: 1.453267	Acc: 25.296381%
  Epoch: 0 [6991/25632/42265 (61%)]	Loss: 1.424821	Acc: 27.274501%
  Epoch: 0 [9146/32032/42265 (76%)]	Loss: 1.501116	Acc: 28.552697%
  Ep

  Epoch: 13 [22/32/42265 (0%)]	Loss: 0.471111	Acc: 68.750000%
  Epoch: 13 [4264/6432/42265 (15%)]	Loss: 0.660116	Acc: 66.293532%
  Epoch: 13 [8539/12832/42265 (30%)]	Loss: 0.446463	Acc: 66.544576%
  Epoch: 13 [12751/19232/42265 (45%)]	Loss: 0.483032	Acc: 66.300957%
  Epoch: 13 [16994/25632/42265 (61%)]	Loss: 0.767807	Acc: 66.299938%
  Epoch: 13 [21271/32032/42265 (76%)]	Loss: 1.082542	Acc: 66.405470%
  Epoch: 13 [25508/38432/42265 (91%)]	Loss: 0.762226	Acc: 66.371774%
Epoch: 13	Loss: 0.614926	Acc: 66.426121
Validation: acc: 46.250000%	loss: 0.075889
Best: 3 @ 49.218750% -> epoch target 70
  Epoch: 14 [24/32/42265 (0%)]	Loss: 0.623733	Acc: 75.000000%
  Epoch: 14 [4277/6432/42265 (15%)]	Loss: 0.476279	Acc: 66.495647%
  Epoch: 14 [8539/12832/42265 (30%)]	Loss: 0.650890	Acc: 66.544576%
  Epoch: 14 [12950/19232/42265 (45%)]	Loss: 0.625495	Acc: 67.335691%
  Epoch: 14 [17306/25632/42265 (61%)]	Loss: 0.676010	Acc: 67.517166%
  Epoch: 14 [21616/32032/42265 (76%)]	Loss: 0.489602	Acc: 67.482517%


Epoch: 26	Loss: 0.555826	Acc: 68.529516
Validation: acc: 45.312500%	loss: 0.096714
Best: 3 @ 49.218750% -> epoch target 70
  Epoch: 27 [20/32/42265 (0%)]	Loss: 0.562851	Acc: 62.500000%
  Epoch: 27 [4386/6432/42265 (15%)]	Loss: 0.504892	Acc: 68.190299%
  Epoch: 27 [8804/12832/42265 (30%)]	Loss: 0.345485	Acc: 68.609726%
  Epoch: 27 [13214/19232/42265 (45%)]	Loss: 0.504279	Acc: 68.708403%
  Epoch: 27 [17632/25632/42265 (61%)]	Loss: 0.500457	Acc: 68.789014%
  Epoch: 27 [22046/32032/42265 (76%)]	Loss: 0.549556	Acc: 68.824925%
  Epoch: 27 [26433/38432/42265 (91%)]	Loss: 0.511310	Acc: 68.778622%
Epoch: 27	Loss: 0.546890	Acc: 68.697504
Validation: acc: 44.843750%	loss: 0.108444
Best: 3 @ 49.218750% -> epoch target 70
  Epoch: 28 [23/32/42265 (0%)]	Loss: 0.507494	Acc: 71.875000%
  Epoch: 28 [4435/6432/42265 (15%)]	Loss: 0.741515	Acc: 68.952114%
  Epoch: 28 [8785/12832/42265 (30%)]	Loss: 0.724431	Acc: 68.461658%
  Epoch: 28 [13149/19232/42265 (45%)]	Loss: 0.608922	Acc: 68.370424%
  Epoch: 28 [17

  Epoch: 40 [22076/32032/42265 (76%)]	Loss: 0.299613	Acc: 68.918581%
  Epoch: 40 [26480/38432/42265 (91%)]	Loss: 0.527142	Acc: 68.900916%
Epoch: 40	Loss: 0.537157	Acc: 68.744824
Validation: acc: 46.562500%	loss: 0.109840
Best: 3 @ 49.218750% -> epoch target 70
  Epoch: 41 [23/32/42265 (0%)]	Loss: 0.410600	Acc: 71.875000%
  Epoch: 41 [4385/6432/42265 (15%)]	Loss: 0.482584	Acc: 68.174751%
  Epoch: 41 [8800/12832/42265 (30%)]	Loss: 0.528817	Acc: 68.578554%
  Epoch: 41 [13170/19232/42265 (45%)]	Loss: 0.499963	Acc: 68.479617%
  Epoch: 41 [17613/25632/42265 (61%)]	Loss: 0.595233	Acc: 68.714888%
  Epoch: 41 [22032/32032/42265 (76%)]	Loss: 0.386018	Acc: 68.781219%
  Epoch: 41 [26457/38432/42265 (91%)]	Loss: 0.484958	Acc: 68.841070%
Epoch: 41	Loss: 0.539372	Acc: 68.900982
Validation: acc: 43.593750%	loss: 0.128406
Best: 3 @ 49.218750% -> epoch target 70
  Epoch: 42 [22/32/42265 (0%)]	Loss: 0.433322	Acc: 68.750000%
  Epoch: 42 [4406/6432/42265 (15%)]	Loss: 0.512850	Acc: 68.501244%
  Epoch: 42 [8

  Epoch: 54 [13271/19232/42265 (45%)]	Loss: 0.379392	Acc: 69.004784%
  Epoch: 54 [17714/25632/42265 (61%)]	Loss: 0.498703	Acc: 69.108926%
  Epoch: 54 [22084/32032/42265 (76%)]	Loss: 0.431875	Acc: 68.943556%
  Epoch: 54 [26421/38432/42265 (91%)]	Loss: 0.655218	Acc: 68.747398%
Epoch: 54	Loss: 0.535341	Acc: 68.704602
Validation: acc: 45.156250%	loss: 0.134231
Best: 3 @ 49.218750% -> epoch target 70
  Epoch: 55 [25/32/42265 (0%)]	Loss: 0.457319	Acc: 78.125000%
  Epoch: 55 [4425/6432/42265 (15%)]	Loss: 0.598804	Acc: 68.796642%
  Epoch: 55 [8865/12832/42265 (30%)]	Loss: 0.402121	Acc: 69.085100%
  Epoch: 55 [13273/19232/42265 (45%)]	Loss: 0.687369	Acc: 69.015183%
  Epoch: 55 [17774/25632/42265 (61%)]	Loss: 0.379911	Acc: 69.343009%
  Epoch: 55 [22144/32032/42265 (76%)]	Loss: 0.701374	Acc: 69.130869%
  Epoch: 55 [26585/38432/42265 (91%)]	Loss: 0.434236	Acc: 69.174126%
Epoch: 55	Loss: 0.538261	Acc: 69.113924
Validation: acc: 46.406250%	loss: 0.116474
Best: 3 @ 49.218750% -> epoch target 70
  Epo

  Epoch: 68 [4409/6432/42265 (15%)]	Loss: 0.372409	Acc: 68.547886%
  Epoch: 68 [8920/12832/42265 (30%)]	Loss: 0.558472	Acc: 69.513716%
  Epoch: 68 [13316/19232/42265 (45%)]	Loss: 0.563676	Acc: 69.238769%
  Epoch: 68 [17719/25632/42265 (61%)]	Loss: 0.594350	Acc: 69.128433%
  Epoch: 68 [22136/32032/42265 (76%)]	Loss: 0.482071	Acc: 69.105894%
  Epoch: 68 [26633/38432/42265 (91%)]	Loss: 0.425994	Acc: 69.299022%
Epoch: 68	Loss: 0.525689	Acc: 69.265350
Validation: acc: 44.531250%	loss: 0.130891
Best: 58 @ 49.375000% -> epoch target 70
  Epoch: 69 [21/32/42265 (0%)]	Loss: 0.596304	Acc: 65.625000%
  Epoch: 69 [4423/6432/42265 (15%)]	Loss: 0.557352	Acc: 68.765547%
  Epoch: 69 [8861/12832/42265 (30%)]	Loss: 0.538384	Acc: 69.053928%
  Epoch: 69 [13195/19232/42265 (45%)]	Loss: 0.522031	Acc: 68.609609%
  Epoch: 69 [17629/25632/42265 (61%)]	Loss: 1.336019	Acc: 68.777310%
  Epoch: 69 [22002/32032/42265 (76%)]	Loss: 0.520371	Acc: 68.687562%
  Epoch: 69 [26416/38432/42265 (91%)]	Loss: 0.514656	Acc: 68.

<Figure size 640x480 with 1 Axes>

['Apis_mellifera', 'Bombus_impatiens', 'Bombus_auricomus', 'Bombus_bimaculatus', 'Bombus_griseocollis']
[61.71875 60.9375  15.625   39.84375 45.3125 ]
